In [68]:
!pip install -q pyspark
!wget -q 'https://drive.google.com/uc?export=download&id=1QkrHJxCU7tonEjrQClTn-30vE93Q8fGh' -O 'Sample.txt'




In [72]:


from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window


conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

input_file = sc.textFile('Sample.txt')

#Using sliding window algorithm
window_size = 6

def sliding_window(iterable,window_size):
  windows = []                               #list to store the generated window
  for i in range(len(iterable) - window_size +  1):
    window=[]                               #list to store the current window
    for j in range(window_size):
      window.append(iterable[i+j])
    windows.append(window)
  return windows


try:
  sliding_rdd = input_file.flatMap(lambda line: sliding_window(line.split(),window_size))
  keyPairs = sliding_rdd.map(lambda window: (' '.join(window), 1))
  countKeys = keyPairs.reduceByKey(lambda a, b: a + b)
  countsDF = sqlContext.createDataFrame(countKeys).withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')
  countsDF.toPandas().to_csv('counts.csv', index=False)
  countsDF.show()
except Exception as e:
  print("AN ERROR OCCURED: ",e)
finally:
  sc.stop()





+--------------------+-----+
|                Word|Count|
+--------------------+-----+
|The Project Guten...|    1|
|Project Gutenberg...|    2|
|This eBook is for...|    2|
|eBook is for the ...|    2|
|is for the use of...|    2|
|use of anyone any...|    2|
|anywhere at no co...|    2|
|almost no restric...|    2|
|no restrictions w...|    2|
|whatsoever. You m...|    2|
|You may copy it, ...|    2|
|it under the term...|    2|
|under the terms o...|    2|
|this eBook or onl...|    2|
|** This is a COPY...|    1|
|This is a COPYRIG...|    1|
|is a COPYRIGHTED ...|    1|
|COPYRIGHTED Proje...|    1|
|** Please follow ...|    1|
|Please follow the...|    1|
+--------------------+-----+
only showing top 20 rows

